In [ ]:
import numpy as np
from connect_four import Game, GameType

In [ ]:
game_init = np.array(
    [
        [2, 2, 2, 1, 0, 1, 0],
        [2, 1, 1, 1, 0, 2, 0],
        [1, 2, 2, 2, 0, 1, 0],
        [2, 1, 1, 1, 0, 2, 0],
        [1, 1, 1, 2, 0, 2, 0],
        [2, 2, 1, 2, 0, 1, 0],
    ]
)

In [ ]:
game = Game(game_state=None, game_type=GameType.MCTS_VS_RANDOM, mcts_maxiter=50)

In [ ]:
game.play()